In [4]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model
## Dense layer 만 써서 RNN 구현
from tensorflow.keras.layers import Dense
## activation fuction
from tensorflow.keras.activations import tanh

import time

In [5]:
!python -m pip show tensorflow


Name: tensorflow
Version: 2.10.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\snuai\anaconda3\lib\site-packages
Requires: numpy, protobuf, tensorflow-io-gcs-filesystem, flatbuffers, wrapt, absl-py, astunparse, termcolor, tensorflow-estimator, grpcio, gast, opt-einsum, packaging, libclang, google-pasta, six, h5py, typing-extensions, tensorboard, keras-preprocessing, setuptools, keras
Required-by: 


In [7]:
def data_generator(n, seq_length=6):
    """
    Args:
        seq_length: Length of the adding problem data
        n: # of data in the set
    """
    x_num = np.random.uniform(0, 1, (n, 1, seq_length))
    x_mask = np.zeros([n, 1, seq_length])
    y = np.zeros([n, 1], dtype=np.float32)
    for i in range(n):
        positions = np.random.choice(seq_length, size=2, replace=False)
        x_mask[i, 0, positions[0]] = 1
        x_mask[i, 0, positions[1]] = 1
        y[i, 0] = x_num[i, 0, positions[0]] + x_num[i, 0, positions[1]]
    x = np.concatenate((x_num, x_mask), axis=1)
    x = np.transpose(x, (0, 2, 1))
    # x.shape = (n,seq_len,2), y.shape=(n,1)
    return x, y

In [8]:
## col 0 : 더할 값 
## col 1 : 더할건지 말건지 결정

print(data_generator(1))


(array([[[0.18503954, 0.        ],
        [0.0458993 , 1.        ],
        [0.64567083, 0.        ],
        [0.50024285, 0.        ],
        [0.41081737, 0.        ],
        [0.07546722, 1.        ]]]), array([[0.12136652]], dtype=float32))


In [9]:
seq_length = 6 

print("Producing data...")
train_data, train_labels = data_generator(100000, seq_length)
print("train_data.shape:", train_data.shape, ", train_labels.shape:", train_labels.shape)

## numpy 를 tensorflow 에서 사용하는 객체로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))

test_data, test_labels = data_generator(1000, seq_length)
test_data, test_labels = tf.convert_to_tensor(test_data), tf.convert_to_tensor(test_labels)

Producing data...
train_data.shape: (100000, 6, 2) , train_labels.shape: (100000, 1)
